In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
df_train = pd.read_csv('train_final.csv', index_col = False)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,10
1,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,10
2,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,10
3,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,10
4,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,10


In [3]:
labels = df_train[['784']]

In [4]:
df_train.drop(df_train.columns[[784]], axis=1, inplace=True)

In [5]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
1,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
2,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
3,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
4,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0


In [6]:
np.random.seed(1212)
import keras 
from keras.models import Model
from keras.layers import *
from keras import optimizers
from keras.layers import Input, Dense, Flatten, Dropout
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.models import model_from_json

In [7]:
df_train.shape

(157485, 784)

In [8]:
temp=df_train.to_numpy()

In [9]:
X_train = temp.reshape(temp.shape[0], 28, 28, 1)

In [10]:
X_train.shape

(157485, 28, 28, 1)

In [11]:
labels = np.array(labels)

In [12]:
from keras.utils.np_utils import to_categorical
cat = to_categorical(labels, num_classes = 14)

In [13]:
l = []
for i in range(157485):
    l.append(np.array(df_train[i:i+1]).reshape(28,28,1))

In [14]:
np.random.seed(7)

In [15]:
#Building and Compiling Model
model = Sequential()
model.add(Conv2D(32,(3,3), input_shape=(28,28,1), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(15, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(14, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
#model training
model.fit(X_train, cat, epochs=10, batch_size=200, shuffle=True, verbose=1)

Epoch 1/10
788/788 [==============================] - 84s 105ms/step - loss: 0.4079 - accuracy: 0.9037
Epoch 2/10
788/788 [==============================] - 78s 99ms/step - loss: 0.0828 - accuracy: 0.9773
Epoch 3/10
788/788 [==============================] - 88s 112ms/step - loss: 0.0549 - accuracy: 0.9849
Epoch 4/10
788/788 [==============================] - 83s 105ms/step - loss: 0.0436 - accuracy: 0.9878
Epoch 5/10
788/788 [==============================] - 76s 97ms/step - loss: 0.0352 - accuracy: 0.9897
Epoch 6/10
788/788 [==============================] - 78s 99ms/step - loss: 0.0297 - accuracy: 0.9911
Epoch 7/10
788/788 [==============================] - 127s 162ms/step - loss: 0.0268 - accuracy: 0.9919
Epoch 8/10
788/788 [==============================] - 77s 97ms/step - loss: 0.0227 - accuracy: 0.9930
Epoch 9/10
788/788 [==============================] - 77s 97ms/step - loss: 0.0218 - accuracy: 0.9933
Epoch 10/10
788/788 [==============================] - 160s 203ms/step - loss

In [17]:
#model saving
model_json = model.to_json()
with open("model_final.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model_final.h5")

In [18]:
#model loading
json_file = open('model_final.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_final.h5")

In [19]:
print(7)

7


In [20]:
import cv2
img = cv2.imread('C:/Users/USER/Downloads/test.jpg',cv2.IMREAD_GRAYSCALE)
if img is not None:
    #images.append(img)
    img=~img
    ret,thresh=cv2.threshold(img,127,255,cv2.THRESH_BINARY)
    ctrs,ret=cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    cnt=sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
    w=int(28)
    h=int(28)
    train_data=[]
    print(len(cnt))
    rects=[]
    for c in cnt :
        x,y,w,h= cv2.boundingRect(c)
        rect=[x,y,w,h]
        rects.append(rect)
    print(rects)
    bool_rect=[]
    for r in rects:
        l=[]
        for rec in rects:
            flag=0
            if rec!=r:
                if r[0]<(rec[0]+rec[2]+10) and rec[0]<(r[0]+r[2]+10) and r[1]<(rec[1]+rec[3]+10) and rec[1]<(r[1]+r[3]+10):
                    flag=1
                l.append(flag)            
            if rec==r:
                l.append(0)
        bool_rect.append(l)
    print(bool_rect)
    dump_rect=[]
    for i in range(0,len(cnt)):
        for j in range(0,len(cnt)):
            if bool_rect[i][j]==1:
                area1=rects[i][2]*rects[i][3]
                area2=rects[j][2]*rects[j][3]
                if(area1==min(area1,area2)):
                    dump_rect.append(rects[i])
    print(len(dump_rect)) 
    final_rect=[i for i in rects if i not in dump_rect]
    print(final_rect)
    for r in final_rect:
        x=r[0]
        y=r[1]
        w=r[2]
        h=r[3]
        im_crop =thresh[y:y+h+10,x:x+w+10]
        

        im_resize = cv2.resize(im_crop,(28,28))


        im_resize=np.reshape(im_resize,(1,28,28))
        train_data.append(im_resize)

3
[[17, 20, 118, 205], [245, 71, 154, 144], [481, 58, 170, 199]]
[[0, 0, 0], [0, 0, 0], [0, 0, 0]]
0
[[17, 20, 118, 205], [245, 71, 154, 144], [481, 58, 170, 199]]


In [21]:
s=''
for i in range(len(train_data)):
    train_data[i]=np.array(train_data[i])
    train_data[i]=train_data[i].reshape(1,28,28,1)
#     result=loaded_model.predict(train_data[i])
    result=np.argmax(loaded_model.predict(train_data[i]), axis=-1)
    if(result[0]==10):
        s=s+'-'
    if(result[0]==11):
        s=s+'+' 
    if(result[0]==12):
        s=s+'*' 
    if(result[0]==13):
        s=s+'/'   
    if(result[0]==10):
        s=s+'-' 
    if(result[0]==1):
        s=s+'1' 
    if(result[0]==2):
        s=s+'2' 
    if(result[0]==3):
        s=s+'3'
    if(result[0]==4):
        s=s+'4' 
    if(result[0]==5):
        s=s+'5' 
    if(result[0]==6):
        s=s+'6' 
    if(result[0]==7):
        s=s+'7' 
    if(result[0]==8):
        s=s+'8'
    if(result[0]==9):
        s=s+'9'     
print(s)   

1/1 [==============================] - 0s 31ms/step
7+3


In [22]:
eval(s)

10